Датасет взят с кэгла. Задача классификации, определить пойдет завтра дождь или нет

https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import dill

from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler



In [2]:
data = pd.read_csv('weatherAUS.csv', sep=',')

In [3]:
# преобразуем target

data.loc[data['RainTomorrow'] == 'No', 'RainTomorrow'] = 0
data.loc[data['RainTomorrow'] == 'Yes', 'RainTomorrow'] = 1

# обработаем признак RainToday

data['RainToday'].fillna(data['RainToday'].mode()[0], inplace=True)

# переведем признак RainToday в числовой
data.loc[data['RainToday'] == 'No', 'RainToday'] = 0
data.loc[data['RainToday'] == 'Yes', 'RainToday'] = 1
data = data.astype({"RainToday": int})

In [4]:
df = data.loc[~data['RainTomorrow'].isna()]
df_test = data.loc[data['RainTomorrow'].isna()]

In [5]:
df = df.astype({"RainTomorrow": int})

In [6]:
df_test.drop('RainTomorrow', axis=1, inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
df.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0,0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0,0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0,0


In [8]:
target_name = 'RainTomorrow'
num_feature_names = ['MinTemp','MaxTemp', 'Rainfall']

In [9]:
df[num_feature_names].head(3)

,MinTemp,MaxTemp,Rainfall
0,13.4,22.9,0.6
1,7.4,25.1,0.0
2,12.9,25.7,0.0


обработаю пропуски

In [10]:
# обработаем пропущенные значения числовых признаков в X_train

for col in df[num_feature_names]:
    col_median = df[col].median()
    df[col].fillna(col_median, inplace=True) 

for col in df_test[num_feature_names]:
    col_median = df_test[col].median()
    df_test[col].fillna(col_median, inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


подготовил датасеты для деление на выборки

In [11]:
X = df[num_feature_names]
y = df[target_name]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)
 
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((99535, 3), (42658, 3), (99535,), (42658,))

In [12]:
#save test
X_valid.to_csv("X_valid.csv", index=None)
y_valid.to_csv("y_valid.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

на данном этапе будут рассматриваться только численные фичи. Опишем классы для всех преобразований

In [13]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
     Преобразователь для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Преобразователь для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований.
    Использовать в числовых столбцах данных
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [14]:
MinTemp =  Pipeline([('selector', NumberSelector(key='MinTemp')),
                     ('standard', StandardScaler()),
                    ])

MaxTemp =  Pipeline([('selector', NumberSelector(key='MaxTemp')),
                     ('standard', StandardScaler()),
                    ])

Rainfall =  Pipeline([('selector', NumberSelector(key='Rainfall')),
                      ('standard', StandardScaler()),
                     ])

In [15]:
feats = FeatureUnion([('MinTemp', MinTemp),
                      ('MaxTemp', MaxTemp),
                      ('Rainfall', Rainfall)])
feature_processing = Pipeline([('feats', feats)])

In [16]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('MinTemp',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='MinTemp')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('MaxTemp',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='MaxTemp')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('Rainfall',
                                                 Pipeline(steps=[('selector',
                            

In [17]:
y_pred = pipeline.predict(X_valid)
pd.Series(y_pred).value_counts()

0    35625
1     7033
dtype: int64

In [18]:
roc_auc = roc_auc_score(y_valid, y_pred)
logloss = log_loss(y_valid, y_pred)
print(f"roc_auc = {round(roc_auc, 2)}\nlogloss = {round(logloss, 2)}")

roc_auc = 0.61
logloss = 8.22


In [19]:
with open("RandomForestClassifier.dill", "wb") as f:
    dill.dump(pipeline, f)